In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import FileVideoStream
import numpy as np
import imutils
import pickle
import cv2
import os
import csv
import pandas as pd

In [2]:
# def detect_and_predict_mask(frame, faceNet, maskNet):
def detect_liveness(frame, faceNet, livenessNet):
	# grab the dimensions of the frame and then construct a blob
	# from it
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
		(104.0, 177.0, 123.0))

	# pass the blob through the network and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the detection
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
			face = frame[startY:endY, startX:endX]
			if face.any():
				# face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
				face = cv2.resize(face, (32, 32))
				# face = img_to_array(face)
				# face = preprocess_input(face)

				# add the face and bounding boxes to their respective
				# lists
				faces.append(face)
				locs.append((startX, startY, endX, endY))

	# only make a predictions if at least one face was detected
	if len(faces) > 0:
		# label = le.classes_[j]
		# for faster inference we'll make batch predictions on *all*
		# faces at the same time rather than one-by-one predictions
		# in the above `for` loop
		faces = np.array(faces, dtype="float") / 255.0
		preds = livenessNet.predict(faces, batch_size=32)

	# return a 2-tuple of the face locations and their corresponding
	# locations
	return (locs, preds)

In [3]:
# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
face = "D:/ZaloAI/LIVENESS/model/face_detection"
prototxtPath =  os.path.sep.join([face, "deploy.prototxt"])
weightsPath = os.path.sep.join([face,
	"res10_300x300_ssd_iter_140000.caffemodel"])
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

[INFO] loading face detector model...


In [4]:
# loading the liveness detecting module that was trained in the training python script
model_path = "liveness.model"
le_path = "le.pickle"
print("loading the liveness detector")
livenessNet = load_model(model_path)
le = pickle.loads(open(le_path, "rb").read())
print(le.classes_)

loading the liveness detector
['fake' 'real']


In [6]:
source = r'D:/ZaloAI/LIVENESS/public_test/public/videos'
data = [] 
for filename in os.scandir(source):
	name = filename.name
	score = 0
	count = 0
	min_score = 1.0
	max_score = 0.0

	# initialize the video stream and allow the camera sensor to warm up
	vs = FileVideoStream(filename.path).start()

	# loop over the frames from the video stream
	while True:
		# grab the frame from the threaded video stream and resize it
		# to have a maximum width of 400 pixels
		frame = vs.read()
		if not vs.more():
			break
		frame = imutils.resize(frame, width=400)

		# detect faces in the frame and determine if they are wearing a
		# face mask or not
		(locs, preds) = detect_liveness(frame, faceNet, livenessNet)
		if len(locs) < 1: 
			continue
		loc = locs[0]
		pred = preds[0]
		
		# loop over the detected face locations and their corresponding
		# locations
		# for i in range(len(locs)):
		# unpack the bounding box and predictions
		(startX, startY, endX, endY) = loc
		(fake, real) = pred
		score += real
		count += 1
		min_score

		# determine the class label and color we'll use to draw
		# the bounding box and text
			
		# # include the probability in the label
		# label = "Liveness"
		# label = "{}: {:.2f}%".format(label, real * 100)
		# color = (0, 255, 0)

		# # display the label and bounding box rectangle on the output
		# # frame
		# cv2.putText(frame, label, (startX, startY - 10),
		# 	cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		# cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

		# # show the output frame
		# cv2.imshow("Frame", frame)
		# key = cv2.waitKey(1) & 0xFF

		# # if the `q` key was pressed, break from the loop
		# if key == ord("q"):
		# 	break
	
	if count == 0:
		count = 1
	score /= count
	data.append([name, score])
	
	# do a bit of cleanup
	# cv2.destroyAllWindows()
	vs.stop()

In [8]:
header = ['fname', 'liveness_score']
data = pd.DataFrame(data, columns = header)
data.to_csv('Predict.csv', index=False)